In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import norm,t

# Import data
def getData(stocks, start, end):
    stockData = yf.download(stocks, start=start, end=end)
    stockData = stockData['Close']
    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return returns, meanReturns, covMatrix

# Portfolio Performance
def portfolioPerformance(weights, meanReturns, covMatrix, Time):
    returns = np.sum(meanReturns*weights)*Time
    std = np.sqrt( np.dot(weights.T, np.dot(covMatrix, weights)) ) * np.sqrt(Time)
    return returns, std

stockList = ['INFY', 'TCS', 'ITC', 'RELIANCE', 'HINDUNILVR', 'ALKEM']
stocks = [stock+'.NS' for stock in stockList]
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=800)

returns, meanReturns, covMatrix = getData(stocks, start=startDate, end=endDate)
returns = returns.dropna()
weights = np.random.random(len(returns.columns))
weights /= np.sum(weights)
returns['Portfolio'] = returns.dot(weights)
alpha = 5

[*********************100%***********************]  6 of 6 completed


In [2]:
#Historical method

#VaR 
def historicalVaR(returns,alpha=5):
    """
    Read in the pandas series of returns
    Output the percentile of the distribution at given alpha confidence level 
    """
    if isinstance(returns,pd.Series):
        return np.percentile(returns,alpha)
    elif isinstance(returns, pd.DataFrame):
        return returns.aggregate(historicalVaR,alpha=5)
    else:
        raise TypeError('Expected returns to be dataframe or series')
        
        
# Expected shortfall
def historicalCVaR(returns,alpha=5):
    """
    Read in the pandas series of returns
    Output the percentile of the distribution at given alpha confidence level 
    """
    if isinstance(returns,pd.Series):
        belowVaR = returns <= historicalVaR(returns,alpha=alpha)
        return returns[belowVaR].mean()
    elif isinstance(returns, pd.DataFrame):
        return returns.aggregate(historicalCVaR,alpha=5)
    else:
        raise TypeError('Expected returns to be dataframe or series')
        
#1day
Time = 1

VaR = -historicalVaR(returns['Portfolio'],alpha=5)
CVaR = -historicalCVaR(returns['Portfolio'], alpha=5)
pRet, pStd = portfolioPerformance(weights,meanReturns,covMatrix,Time)

        

    
#Parametric

#VaR
def var_parametric(portofolioReturns, portfolioStd, distribution='normal', alpha=5, dof=6):
    # because the distribution is symmetric
    if distribution == 'normal':
        VaR = norm.ppf(1-alpha/100)*portfolioStd - portofolioReturns
    elif distribution == 't-distribution':
        nu = dof
        VaR = np.sqrt((nu-2)/nu) * t.ppf(1-alpha/100, nu) * portfolioStd - portofolioReturns
    else:
        raise TypeError("Expected distribution type 'normal'/'t-distribution'")
    return VaR

#Expected Shortfall
def cvar_parametric(portofolioReturns, portfolioStd, distribution='normal', alpha=5, dof=6):
    if distribution == 'normal':
        CVaR = (alpha/100)**-1 * norm.pdf(norm.ppf(alpha/100))*portfolioStd - portofolioReturns
    elif distribution == 't-distribution':
        nu = dof
        xanu = t.ppf(alpha/100, nu)
        CVaR = -1/(alpha/100) * (1-nu)**(-1) * (nu-2+xanu**2) * t.pdf(xanu, nu) * portfolioStd - portofolioReturns
    else:
        raise TypeError("Expected distribution type 'normal'/'t-distribution'")
    return CVaR
        
normVaR = var_parametric(pRet,pStd)
normCVaR = cvar_parametric(pRet,pStd)
tVaR = var_parametric(pRet, pStd, distribution='t-distribution')
tCVaR = cvar_parametric(pRet, pStd, distribution='t-distribution')






#VaR and Expected shotfall Calculation

InitialInvestment = 100000
print ("\033[1m" + "VaR and CVaR Historical Method" + "\033[0m")
print('Expected port ret: ', round(pRet*InitialInvestment,2))
print('VaR at Risk 95th CI: ', round(VaR*InitialInvestment,2))
print('Conditional VaR 95th CI: ', round(CVaR*InitialInvestment,2))
print('\n')

print ("\033[1m" + "VaR and CVaR Parametric Method" + "\033[0m")
print('normal VaR 95th CI: ',round(InitialInvestment*normVaR,2))
print('normal CVaR 95th CI: ',round(InitialInvestment*normCVaR,2))
print('t-dist VaR 95th CI: ',round(InitialInvestment*tVaR,2))
print('t-dist CVaR 95th CI: ',round(InitialInvestment*tCVaR,2))
print('\n')


VaR and CVaR Historical Method
Expected port ret:  99.34
VaR at Risk 95th CI:  1685.63
Conditional VaR 95th CI:  2956.09


VaR and CVaR Parametric Method
normal VaR 95th CI:  2158.47
normal CVaR 95th CI:  2732.04
t-dist VaR 95th CI:  2078.5
t-dist CVaR 95th CI:  2860.32


